This file is the short notebook to make the graph COSI version on SWAT dataset

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from timeit import default_timer as timer
import time
import re
import Tools_first_step as tfs
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.mode.chained_assignment = None

In [ ]:
threashold=0.7
L=0.5

In [ ]:
path_dataset="/path/preprocessed_training.csv"
dataset="swat"

In [ ]:
df=pd.read_csv(path_dataset)
columns=df.columns

In [ ]:
print(df.shape)
col_to_keep=tfs.rmv_Unnamed(df.columns)
df=df[col_to_keep]
print(df.shape)
df.head()

In [ ]:
data_description_path="/path/Swat/edge_description.txt"
def parse_txt_to_dict(file_path):
    data_dict = {}
    with open(file_path, 'r') as file:
        next(file)
        for line in file:
            if not line.strip():
                continue  # skip empty lines
            parts = line.strip().split(' ', 1)  # split only at the first space
            #print(parts)
            if len(parts) == 2:
                key = int(parts[0])  # assuming the key is always an integer
                value = parts[1]
                data_dict[key] = value
    return data_dict

dic_sensor_description=parse_txt_to_dict(data_description_path)
print(dic_sensor_description)

In [ ]:
new_col_names=[]
for k in dic_sensor_description.keys():
    new_col_names.append(dic_sensor_description[k])

df.columns=new_col_names
df.head()

In [ ]:
def get_word_from_dic_swat(dictionary_swat):
    Words=[]
    for k in dictionary_swat.keys():
        features_full_desc=[]
        features_full_desc=re.split("[- ; . ( )]", dictionary_swat[k])
        new_f=[s for s in features_full_desc if s != '']
        #final_f= [s for s in features_full_desc if s.strip() != '' and not s.strip().isdigit()]

        print(new_f)
        Words.append(new_f)
    
    return Words

words_swat=get_word_from_dic_swat(dic_sensor_description)

In [ ]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

# Load pretrained model (download if needed: GoogleNews-vectors-negative300.bin)
pretrained_model = KeyedVectors.load_word2vec_format('/path/GoogleNews-vectors-negative300.bin', binary=True)

# Convert KeyedVectors to a Word2Vec instance for further training
model = Word2Vec(vector_size=300, min_count=1)
model.build_vocab([list(pretrained_model.key_to_index.keys())], update=False)
model.wv.vectors = pretrained_model.vectors


In [ ]:
model.build_vocab(words_swat, update=True)

In [ ]:
model.train(words_swat, total_examples=len(words_swat), epochs=10)

In [ ]:
name_columns=df.columns
cosine_similarities,vectores_places=tfs.score_similarity_current(name_columns, model, dt=dataset)

In [ ]:
print(cosine_similarities)

In [ ]:
name_columns=df.columns
print(tfs.estimate_total_number_of_word(name_columns, dt=dataset))
number_complet=tfs.estimate_total_number_of_word(name_columns, dt=dataset)
print(tfs.find_frequence_of_words(name_columns, dt=dataset)[1])

In [ ]:
sequence1 =model.wv['Sodium'] 
sequence2 =model.wv['Sulphate']
similarity = tfs.cosine_similarity(sequence1, sequence2)
print("Cosine Similarity:", similarity)

In [ ]:
xe,ye,ze, all_edges=tfs.get_edges_topK(cosine_similarities,threashold, vectores_places)

In [ ]:
nb=tfs.get_distribution_number_of_edges_per_nodes("",cosine_similarities,threashold, vectores_places)


In [ ]:
tfs.plot_data_pca(cosine_similarities,threashold,vectores_places)

In [ ]:
corr_mat=df.corr()

In [ ]:
corr_mat.index=df.columns
corr_mat.fillna(1, inplace=True)
corr_mat.head()


In [ ]:
adj=tfs.build_adjacency_matrix_v2(all_edges[1:],corr_mat)

In [ ]:
adj=np.array(adj)

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(adj, cmap='copper')
plt.title('Adjacency matrix 25D')
plt.show()

With correlation (COSI classic)

In [ ]:
dico_correlation=tfs.change_into_dico(corr_mat)
My_dictionary=tfs.build_new_score_product(dico_correlation, cosine_similarities) 
My_cosi_5_dic=tfs.build_score_lambda(dico_correlation, cosine_similarities, L)
my_edges_test= tfs.edges_dico(My_dictionary, threashold)
useless1,useless2,useless3, edges_complet=tfs.get_edges_topK(My_dictionary,threashold,vectores_places,"", 200)
useless11,useless22,useless33, edges_complet_cosi5=tfs.get_edges_topK(My_cosi_5_dic,threashold,vectores_places,"", 200)


In [ ]:
tfs.plot_data_pca(My_dictionary,threashold,vectores_places)

In [ ]:
med=tfs.get_medium_number_of_edges(edges_complet_cosi5)
print(f'average number of edges per nodes: {med}')

In [ ]:
adj_cosi=tfs.build_adjacency_matrix_v2(edges_complet[1:],corr_mat)

In [ ]:
adj_cosi_np=np.array(adj_cosi)

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(adj_cosi_np, cmap='copper')
plt.title('Adjacency matrix 25D')
plt.show()

In [ ]:
#np.savetxt("/path/Swat/corr.txt", adj_cosi_np, delimiter=',' )